In [92]:
# to mount my drive for simplicity
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
import pandas as pd
import nltk
import string
from nltk.stem import PorterStemmer

w2v = {}
df = pd.read_csv('drive/MyDrive/dataset_lyrics.csv')

nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
docs_without_punctuation = []
ps = PorterStemmer()

docs_without_stopwords = df['lyrics'].apply(lambda x: ' '.join([w for w in x.split() if w not in (stop_words)]))
for row in docs_without_stopwords:
  docs_without_punctuation.append(row.translate(str.maketrans('','', string.punctuation)))
stemmed = [[ps.stem(w) for w in row.split(" ")] for row in docs_without_punctuation]
print(len(stemmed))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


28372


In [94]:
from gensim.models import Word2Vec
model1 = Word2Vec(stemmed, min_count = 1,size = 10, window = 5, sg=0)
model1.wv.vocab

model2 = Word2Vec(stemmed, min_count = 1,size = 30, window = 10, sg=0)

In [95]:
print(model1.wv['fuck']) # so bekommt man vec zu einem word
print(model2.wv['fuck']) # so bekommt man vec zu einem word

[-6.3880887   0.03469695 -0.26153627 -1.9968396   3.1164758  -0.29495043
  0.10844582  1.7513279   2.90111     2.3267026 ]
[-3.331364    0.46976304  0.21766092 -5.2021303   3.2504268   1.9737006
 -0.4590966   2.8487244   0.11629162  4.174475   -0.26108804 -2.8431518
  3.0733502  -0.8551074   2.180512   -0.37739226  0.974011    2.812875
  2.461448    3.7926676   0.4704313   2.9150212  -0.18501164 -2.864731
 -3.4984462   0.3310644   3.0437663  -4.3429966  -2.0351987   0.4443268 ]


In [96]:
import numpy as np

# avgerage pooling implementation
def avg_pool(doc, model):
    pool = np.array([0.0]*model.vector_size)
    
    for idx, word in enumerate(doc):
      vec = model.wv[word]
      pool += vec[0]

    pool = pool / len(doc)

        
    return pool

# avgerage pool list of wvs
def avg_pool_wvs(wvs):
    pool = np.array([0.0]*len(wvs[0]))
    
    for idx, song in enumerate(wvs):
      pool += song

    pool = pool / len(wvs)
        
    return pool

# cosine similarity implementation
def cosine_sim(a, b):
  dot = np.dot(a, b)
  aNorm = np.linalg.norm(a)
  bNorm = np.linalg.norm(b)

  denom = (aNorm * bNorm)

  if denom > 0.0:
    return dot / denom
  elif dot == 0.0:
    return 0.0

In [97]:
track_to_wv = dict()
artist_to_wv = dict()
artist_to_wvs = dict()
track_to_wv_test = dict()
track_to_lyrics = dict()


def extract_data(model):
  # reset globals 
  global track_to_wv
  global artist_to_wv
  global artist_to_wvs
  global track_to_wv_test
  global track_to_lyrics

  track_to_wv = dict()
  artist_to_wv = dict()
  artist_to_wvs = dict()
  track_to_wv_test = dict()
  track_to_lyrics = dict()

  test = 10000
  cnt = 0

  # split dataset into test, train data
  for index, row in df.iterrows():
    # prepare train data
    if cnt < test:
      docs_without_punctuation = []
      docs_without_stopwords = [w for w in row['lyrics'].split() if w not in (stop_words)]
      docs_without_punctuation = [w for w in docs_without_stopwords if w not in (string.punctuation)]

      stemmed = [[ps.stem(w) for w in roww.split(" ")] for roww in docs_without_punctuation]

      track_to_lyrics[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = stemmed
      track_to_wv[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = avg_pool(stemmed, model)

      if str(row['artist_name']) not in artist_to_wvs:
        artist_to_wvs[str(row['artist_name'])] = []
      artist_to_wvs[str(row['artist_name'])].append(avg_pool(stemmed, model))

    # or prepare test data
    elif cnt >= test:
      docs_without_punctuation = []
      docs_without_stopwords = [w for w in row['lyrics'].split() if w not in (stop_words)]
      docs_without_punctuation = [w for w in docs_without_stopwords if w not in (string.punctuation)]

      stemmed = [[ps.stem(w) for w in roww.split(" ")] for roww in docs_without_punctuation]

      track_to_lyrics[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = stemmed
      track_to_wv_test[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = avg_pool(stemmed, model)
    
    if cnt == int(test*1.2):
      break
    cnt = cnt + 1

  for name, songs in artist_to_wvs.items():
    artist_to_wv[name] = avg_pool_wvs(songs)

extract_data(model1)

print(len(artist_to_wv))
print(artist_to_wv['ricky skaggs'])

2047
[-1.98591426 -1.53492835  1.51768827  0.50054728 -1.98246548 -2.11891772
  0.38500297 -0.37082132  1.76937252 -0.93111584]


In [98]:
# remove stopwords, punctuaction, and do stemming.
# returns stemmed query and average pooled word vector
def preprocess_query(user_query, model):
  user_query_without_stopwords = []
  user_query_without_punctuation = []
  user_query_without_stopwords = [w for w in user_query.split() if w not in (stop_words)]
  user_query_without_punctuation = [w for w in user_query_without_stopwords if w not in (string.punctuation)]

  user_query_stemmed = [[ps.stem(w) for w in roww.split(" ")] for roww in user_query_without_punctuation]

  return user_query_stemmed, avg_pool(user_query_stemmed, model)

In [99]:
# the doors - hello, i love you
the_doors_user_query = "hello i love you"

user_query_stemmed, user_query_stemmed_wv = preprocess_query(the_doors_user_query, model1)

In [100]:
import torch
from torch import nn

# Create model
device = "cpu"

# Define model
class LyricsSimilarityModel1(nn.Module):
    def __init__(self):
        super(LyricsSimilarityModel1, self).__init__()
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(10, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 1)
            
        )

    def forward(self, inputs_):
        x = self.linear(inputs_)
        return x

lyricsSimModel1 = LyricsSimilarityModel1().to(device)
lyricsSimModel1 = lyricsSimModel1.float()

In [106]:
def train(data, user_query_stemmed, model, loss_fn, optimizer):
  model.train()

  for key, item in data.items():
    # predict output using one hot encoding of word

    pred = model(torch.tensor(item).float())

    target = cosine_sim(avg_pool(user_query_stemmed, model1), item)

    #print(pred)
    #print(torch.tensor([target]).float())

    # Compute prediction error
    loss = loss_fn(pred, torch.tensor([target]).float())

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [104]:
def test(data, user_query_stemmed, model, loss_fn, optimizer):
  model.eval()

  threshold = 0.01
  correct = 0

  for key, item in data.items():
    # predict output using one hot encoding of word

    pred = model(torch.tensor(item).float())

    target = cosine_sim(avg_pool(user_query_stemmed, model1), item)

    #print(pred)
    #print(torch.tensor([target]).float())

    # Compute prediction error
    loss = loss_fn(pred, torch.tensor([target]).float())
    #print(loss)
    #print()

    if (target.item() - pred.item()) <= threshold:
      correct += 1
  
  print("accuracy: " + str(correct / len(data.items())))
  return

In [107]:
# Baseline LyricsModel1 

optimizer = torch.optim.Adagrad(lyricsSimModel1.parameters(), lr=0.2)
loss_fn = torch.nn.BCEWithLogitsLoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, user_query_stemmed, lyricsSimModel1, loss_fn, optimizer)
    test(track_to_wv_test, user_query_stemmed, lyricsSimModel1, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
accuracy: 0.584207896051974
Epoch 2
-------------------------------
accuracy: 0.6011994002998501
Epoch 3
-------------------------------
accuracy: 0.6166916541729135
Epoch 4
-------------------------------
accuracy: 0.6211894052973513
Epoch 5
-------------------------------
accuracy: 0.6176911544227887
Done!


In [108]:
# LyricsModel1 with other loss / optimizer

lyricsSimModel1 = LyricsSimilarityModel1().to(device)
lyricsSimModel1 = lyricsSimModel1.float()

# use mseloss because this is not a classification task
# also change optimizer for testing
optimizer = torch.optim.SGD(lyricsSimModel1.parameters(), lr=0.05)
loss_fn = torch.nn.MSELoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, user_query_stemmed, lyricsSimModel1, loss_fn, optimizer)
    test(track_to_wv_test, user_query_stemmed, lyricsSimModel1, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
accuracy: 0.6006996501749126
Epoch 2
-------------------------------
accuracy: 0.6361819090454772
Epoch 3
-------------------------------
accuracy: 0.6731634182908546
Epoch 4
-------------------------------
accuracy: 0.6956521739130435
Epoch 5
-------------------------------
accuracy: 0.7126436781609196
Done!


In [109]:
# Create model
device = "cpu"

# Define 2nd model with more layers
class LyricsSimilarityModel2(nn.Module):
    def __init__(self):
        super(LyricsSimilarityModel2, self).__init__()
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(10, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 1)
        )

    def forward(self, inputs_):
        x = self.linear(inputs_)
        return x

lyricsSimModel2 = LyricsSimilarityModel2().to(device)
lyricsSimModel2 = lyricsSimModel2.float()

In [110]:
# use mseloss because this is not a classification task
# also change optimizer for testing
optimizer = torch.optim.SGD(lyricsSimModel2.parameters(), lr=0.05)
loss_fn = torch.nn.MSELoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, user_query_stemmed, lyricsSimModel2, loss_fn, optimizer)
    test(track_to_wv_test, user_query_stemmed, lyricsSimModel2, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
accuracy: 0.6606696651674163
Epoch 2
-------------------------------
accuracy: 0.7246376811594203
Epoch 3
-------------------------------
accuracy: 0.7486256871564217
Epoch 4
-------------------------------
accuracy: 0.7581209395302348
Epoch 5
-------------------------------
accuracy: 0.7481259370314842
Done!


In [111]:
# Create model
device = "cpu"

# Define 2nd model with more layers
class LyricsSimilarityModel3(nn.Module):
    def __init__(self):
        super(LyricsSimilarityModel3, self).__init__()
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(10, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 30),
            torch.nn.ReLU(),
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 1),
        )

    def forward(self, inputs_):
        x = self.linear(inputs_)
        return x

lyricsSimModel3 = LyricsSimilarityModel3().to(device)
lyricsSimModel3 = lyricsSimModel3.float()

In [112]:
# use mseloss because this is not a classification task
# also change optimizer for testing
optimizer = torch.optim.SGD(lyricsSimModel3.parameters(), lr=0.05)
loss_fn = torch.nn.MSELoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, user_query_stemmed, lyricsSimModel3, loss_fn, optimizer)
    test(track_to_wv_test, user_query_stemmed, lyricsSimModel3, loss_fn, optimizer)
print("Done!")

# LyricsModel3 seems to work fine for now

Epoch 1
-------------------------------
accuracy: 0.7116441779110445
Epoch 2
-------------------------------
accuracy: 0.8170914542728636
Epoch 3
-------------------------------
accuracy: 0.8460769615192404
Epoch 4
-------------------------------
accuracy: 0.8525737131434283
Epoch 5
-------------------------------
accuracy: 0.8785607196401799
Done!


In [113]:
# compare model results with cosine_sim results

# word2vec model results
query = []
for item in user_query_stemmed:
  query.append(item[0])
print(model1.most_similar(query)[:10])

[('liplock', 0.9391372799873352), ('shiiett', 0.9287172555923462), ('darlin', 0.9213064312934875), ('lover', 0.9201908111572266), ('romanc', 0.8944610357284546), ('pullov', 0.8921284079551697), ('hungr', 0.8868628740310669), ('darl', 0.8771656155586243), ('sweetheart', 0.8739891052246094), ('uuuuuh', 0.8730219006538391)]


<ipython-input-113-f7107487c04a>:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  print(model1.most_similar(query)[:10])


In [114]:
# add some more user queries based on song lyrics

# Godsmack - Good day to die
godsmack_user_query = "Today is a good day to die"
user_query_stemmed_godsmack, user_query_stemmed_wv_godsmack = preprocess_query(godsmack_user_query, model1)

# this song is not in the dataset!
# Nirvana - Frances farmer will have her revenge on seattle
nirvana_user_query = "I miss the comfort in being sad"
user_query_stemmed_nirvana, user_query_stemmed_wv_nirvana = preprocess_query(nirvana_user_query, model1)

In [115]:
# put all user queries in a list for simplicity
user_queries = [the_doors_user_query, godsmack_user_query, nirvana_user_query]

In [116]:
optimizer = torch.optim.SGD(lyricsSimModel3.parameters(), lr=0.05)
loss_fn = torch.nn.MSELoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, user_query_stemmed,lyricsSimModel3, loss_fn, optimizer)
    train(track_to_wv, user_query_stemmed_godsmack,lyricsSimModel3, loss_fn, optimizer)
    test(track_to_wv_test, user_query_stemmed, lyricsSimModel3, loss_fn, optimizer)
    test(track_to_wv_test, user_query_stemmed_godsmack, lyricsSimModel3, loss_fn, optimizer)

    # test one query which is not in the dataset and not trained
    test(track_to_wv_test, user_query_stemmed_nirvana, lyricsSimModel3, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
accuracy: 0.9010494752623688
accuracy: 0.8925537231384307
accuracy: 0.5502248875562219
Epoch 2
-------------------------------
accuracy: 0.9005497251374313
accuracy: 0.8825587206396801
accuracy: 0.543728135932034
Epoch 3
-------------------------------
accuracy: 0.9005497251374313
accuracy: 0.8840579710144928
accuracy: 0.5357321339330335
Epoch 4
-------------------------------
accuracy: 0.9005497251374313
accuracy: 0.9015492253873063
accuracy: 0.5367316341829086
Epoch 5
-------------------------------
accuracy: 0.8990504747626187
accuracy: 0.8860569715142429
accuracy: 0.5302348825587206
Done!


In [117]:
# get most similar songs to all given user queries

for q in user_queries:
  q_stemmed, q_stemmed_wv = preprocess_query(q, model1)

  most_sim = dict()
  for name, wv in track_to_wv.items():
    most_sim[name] = cosine_sim(wv, q_stemmed_wv)
  print("The 10 most similar (cosine_sim) songs by lyrics to the user query '" + q + "':")
  print(sorted(most_sim.items(), key=lambda item: item[1], reverse=True)[:10])

The 10 most similar (cosine_sim) songs by lyrics to the user query 'hello i love you':
[("the beach boys i'll bet he's nice 1977", 0.9623505583847757), ('glen campbell dream baby (how long must i dream) 1971', 0.9467735169102647), ('ray price letters have no arms 1957', 0.9450332568784264), ('chubby checker lovely, lovely (loverly, loverly) 1964', 0.9446817436943032), ('mary wells two lovers 1963', 0.9421651415630538), ('johnny burnette dream lover 1961', 0.9395670023796022), ('roy orbison dream 1963', 0.9393750440537207), ('the knack heartbeat 1979', 0.9375160164793015), ('tammy wynette sweet dreams 1964', 0.9367451511400855), ("smokey robinson & the miracles what's so good about goodbye 1962", 0.9362855894948926)]
The 10 most similar (cosine_sim) songs by lyrics to the user query 'Today is a good day to die':
[("merle haggard i'd trade all of my tomorrows 1965", 0.9952810929023288), ('willie nelson yesterday 1966', 0.9840212571552907), ('willie nelson night life 1965', 0.978819941714

In [118]:
# get most similar artists to a given artist
def get_x_most_similar_artists_to(artist_name, x):
  artist_wv = artist_to_wv[artist_name]
  most_sim = dict()
  for name, wv in artist_to_wv.items():
    most_sim[name] = cosine_sim(wv, artist_wv)
  print("The 10 most similar (cosine_sim) songs by lyrics to the user query '" + artist_name + "':")

  return sorted(most_sim.items(), key=lambda item: item[1], reverse=True)[:x]

# test model1
print(get_x_most_similar_artists_to('snoop dogg', 10))

# test model2
extract_data(model2)
print(get_x_most_similar_artists_to('snoop dogg', 10))

The 10 most similar (cosine_sim) songs by lyrics to the user query 'snoop dogg':
[('snoop dogg', 0.9999999999999998), ('jadakiss', 0.9895283614356974), ('transplants', 0.9886533589008788), ('lloyd banks', 0.987871156408182), ('a$ap rocky', 0.9875904124407174), ('a$ap ferg', 0.9875342024620539), ('young money', 0.9863482363074052), ('naughty by nature', 0.9858910599371251), ('jeezy', 0.9852158664050749), ('trina', 0.9840271441712206)]
The 10 most similar (cosine_sim) songs by lyrics to the user query 'snoop dogg':
[('snoop dogg', 1.0), ('a$ap rocky', 0.9623914340861929), ('transplants', 0.9547817345687128), ('young money', 0.9538332105902102), ('jadakiss', 0.9533323327099748), ('jeezy', 0.9532148985129835), ('2pac', 0.9516091952621718), ('outkast', 0.9501447072120921), ('big pun', 0.9491968676869726), ('travis scott', 0.9490251183710146)]
